In [ ]:
from db_connection import db_query, db_query_dataframe, read_query, update_query
import pandas as pd
import numpy as np

DB_SETTINGS = 'host=localhost dbname=addicted \
    user=addicted_r password=addicted_r port=3456'
STORY_ID = 'j2YTBTDFI1'

def get_df_from_db(query_path):
    """Функция для выполнения SQL-запроса, расположенного по указанному адресу."""
    query = read_query(query_path)
    query = update_query(query, params={'story_id' : STORY_ID})
    return db_query_dataframe(DB_SETTINGS, query)

### Извлекаем данные:

- данные о прогрессе пользователей по указанной истории;
- данные о покупках, совершённых при прочтении данной истории;
- данные о времени публикации эпизодов и наличии ранних доступов.

In [394]:
progress = get_df_from_db('sqls/get_progress.sql')
purchases = get_df_from_db('sqls/get_purchases.sql')
episodes = get_df_from_db('sqls/get_episodes.sql')

Database connected
Database connected
Database connected


### Cчитаем значения дополнительных полей в таблице покупок:

- среднее количество ребиллов на iOS;
- среднее количество ребиллов на Android.

In [256]:
purchases['avg_ios_rebills'] = purchases['ios_rebills']/purchases['ios_subs']
purchases['avg_android_rebills'] = purchases['android_rebills']/purchases['android_subs']
purchases.head(3)

,purchaseDate,paid_users,all_purs,ios_subs,ios_rebills,android_subs,android_rebills,early_accesses,total_revenue,ios_subs_revenue,android_subs_revenue,early_access_revenue,avg_ios_rebills,avg_android_rebills
0,2019-12-05,1,1,1,2,0,0,0,340.016,340.016,0,0,2.000000,NaN
1,2019-12-06,7,7,7,12,0,0,0,2040.096,2040.096,0,0,1.714286,NaN
2,2019-12-07,4,4,3,1,1,4,0,358.976,170.008,188.968,0,0.333333,4.0


### Таблицу с информацией об эпизодах оставим как есть:

In [257]:
episodes

,published_at,free_access_at,is_early_access,title,textId,author,ep_num,finished,hasAudio,episodesCount
0,2019-12-05,None,false,Заброшенная школа,ru-abandoned-school,hey_all_nice,1,false,True,10.0
1,2019-12-05,None,false,Заброшенная школа,ru-abandoned-school,hey_all_nice,2,false,True,10.0
2,2019-12-05,None,false,Заброшенная школа,ru-abandoned-school,hey_all_nice,3,false,True,10.0
3,2019-12-12,None,false,Заброшенная школа,ru-abandoned-school,hey_all_nice,4,false,True,10.0
4,2019-12-19,None,false,Заброшенная школа,ru-abandoned-school,hey_all_nice,5,false,True,10.0
5,2019-12-19,2019-12-26,true,Заброшенная школа,ru-abandoned-school,hey_all_nice,6,false,True,10.0
6,2019-12-26,2020-01-02,true,Заброшенная школа,ru-abandoned-school,hey_all_nice,7,false,True,10.0
7,2020-01-02,2020-01-09,true,Заброшенная школа,ru-abandoned-school,hey_all_nice,8,false,True,10.0
8,2020-01-09,2020-01-16,true,Заброшенная школа,ru-abandoned-school,hey_all_nice,9,false,True,10.0
9,2020-01-16,2020-01-23,true,Заброшенная школа,ru-abandoned-school,hey_all_nice,10,true,True,10.0


### Очистим таблицу с прогрессом от мусора:

- удалим записи с датой обновления позже текущей;
- удалим записи с датой обновления раньше публикации истории.

In [258]:
import datetime

story_published_at = episodes[(episodes['ep_num'] == 1)]['published_at'][0]
progress = progress[(progress['updated_at'] <= datetime.date.today())]
progress = progress[(progress['updated_at'] >= story_published_at)].reset_index(drop=True)
progress

,updated_at,fin_eps,users_read
0,2019-12-05,0,1726
1,2019-12-05,1,330
2,2019-12-05,2,252
3,2019-12-05,3,2515
4,2019-12-05,4,1
...,...,...,...
2804,2020-08-26,5,8
2805,2020-08-26,6,3
2806,2020-08-26,7,2
2807,2020-08-26,8,2


### Напишем функцию подсчёта логов по истории

In [259]:
def count_eps_ea_by_day(episodes, pub_days):
    """Функция для подсчета логов: сколько эпизодов опубликовано, а сколько – находятся на раннем доступе
    по датам на протяжении всех дней, в течение которых история публиковалась."""
    rows = []
    eps_published = 0

    for i in range(pub_days+1):
        curr_day = first_ep_date+datetime.timedelta(days=i)
        ea_count = 0
#         print(curr_day, i, n)
        
        for _, row in episodes.iterrows():
            if (curr_day == row['published_at']):
                eps_published = row['ep_num']
            
            if (pd.isnull(row['free_access_at'])):
                continue
            elif (curr_day >= row['published_at'] and 
                  curr_day < row['free_access_at']):
                ea_count+=1
                
        row = {
            'date': curr_day,
            'eps_published': eps_published,
            'ea_count': ea_count
        }
        rows.append(row)
        
    return pd.DataFrame.from_dict(rows)

Вычислим крайние даты публикации эпизодов (с учётом раннего доступа), а также количество дней, в течение которых история публиковалась. После этого применим функцию подсчёта логов.

In [260]:
first_ep_date = episodes['published_at'].min()
last_ep_date = max(episodes['published_at'].max(),
                   episodes['free_access_at'].dropna().max())
pub_days = (last_ep_date - first_ep_date).days
    
ep_info = count_eps_ea_by_day(episodes, pub_days)

Добавим к таблице прогресса столбцы с количеством опубликованных эпизодов (`eps_published`) и количеством эпизодов на раннем доступе (`ea_count`). Заполним значениями для дат, наступивших позже даты публикации последнего эпизода с учётом раннего доступа:
- максимальное количество опубликованных эпизодов – для столбца `eps_published`;
- 0 – для столбца `ea_count`.

Соединим таблицу `progress` с логами (таблицей `ep_info`) и покупками (`purchases`).

In [264]:
def get_merged_data(progress, ep_info, last_ep_date):
    temp_p = progress
    temp_p['eps_published'] = temp_p['updated_at'].apply(lambda x: ep_info['eps_published'].max() 
                                                         if x > last_ep_date else None)
    temp_p['ea_count'] = temp_p['updated_at'].apply(lambda x: 0
                                                    if x > last_ep_date else None)

    data = temp_p.merge(ep_info, left_on='updated_at',
                        right_on='date', how='outer')

    data_1 = data[['eps_published_x', 'ea_count_x']].rename(columns = {
        'eps_published_x': 'eps_published', 'ea_count_x': 'ea_count'
    })
    data_2 = data[['eps_published_y', 'ea_count_y']].rename(columns = {
        'eps_published_y': 'eps_published', 'ea_count_y': 'ea_count'
    })

    merged_eps = data_1.combine(data_2, np.fmax)

    data['eps_published'] = merged_eps['eps_published']
    data['ea_count'] = merged_eps['ea_count']

    data = data[['updated_at', 'fin_eps', 'users_read', 'eps_published', 'ea_count']]
    
    data = data.merge(purchases, 
                      left_on='updated_at', right_on='purchaseDate', 
                      how='outer')
    
    total_start_read_data = data[['updated_at', 'users_read']].groupby(
        'updated_at').sum().reset_index().rename(columns={'users_read': 'users_stop_reading'})
    
    data = data.merge(total_start_read_data, 
                      left_on= 'updated_at', right_on='updated_at', 
                      how='outer').rename(columns={'updated_at': 'date'})
    return data

In [265]:
merged_data = get_merged_data(progress, ep_info, last_ep_date)

In [267]:
merged_data

,date,fin_eps,users_read,eps_published,ea_count,purchaseDate,paid_users,all_purs,ios_subs,ios_rebills,android_subs,android_rebills,early_accesses,total_revenue,ios_subs_revenue,android_subs_revenue,early_access_revenue,avg_ios_rebills,avg_android_rebills,users_stop_reading
0,2019-12-05,0,1726,3.0,0.0,2019-12-05,1.0,1.0,1.0,2.0,0.0,0.0,0.0,340.016,340.016,0,0,2.0,NaN,4826
1,2019-12-05,1,330,3.0,0.0,2019-12-05,1.0,1.0,1.0,2.0,0.0,0.0,0.0,340.016,340.016,0,0,2.0,NaN,4826
2,2019-12-05,2,252,3.0,0.0,2019-12-05,1.0,1.0,1.0,2.0,0.0,0.0,0.0,340.016,340.016,0,0,2.0,NaN,4826
3,2019-12-05,3,2515,3.0,0.0,2019-12-05,1.0,1.0,1.0,2.0,0.0,0.0,0.0,340.016,340.016,0,0,2.0,NaN,4826
4,2019-12-05,4,1,3.0,0.0,2019-12-05,1.0,1.0,1.0,2.0,0.0,0.0,0.0,340.016,340.016,0,0,2.0,NaN,4826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2804,2020-08-26,5,8,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,186
2805,2020-08-26,6,3,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,186
2806,2020-08-26,7,2,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,186
2807,2020-08-26,8,2,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,186


In [313]:
def get_ea_markers(ep_info):
    rows = []
    date = ep_info['date'][0]
    ea_count = ep_info['ea_count'][0]
    r = {'date': date, 'ea_count': ea_count}
    rows.append(r)

    for _, row in ep_info.iterrows():
        if (ea_count != row['ea_count']):
            date = row['date']
            ea_count = row['ea_count']
            new_r = {'date': date, 'ea_count': ea_count}
            rows.append(new_r)

    return pd.DataFrame.from_dict(rows)

In [328]:
def published_eps_trace(height):
    markers = episodes[['published_at', 'ep_num']].groupby('published_at').max().reset_index()
    markers['height'] = height
    
    return go.Scatter(x=markers['published_at'], 
                         y=markers['height'],
                         mode="markers+text",
                         name="Episodes Published",
                         text=markers['ep_num'],
                         textposition="bottom center")

In [330]:
def ea_markers_trace(height):
    ea_markers = get_ea_markers(ep_info)
    ea_markers['height'] = height
    
    return go.Scatter(x=ea_markers['date'], 
                         y=ea_markers['height'],
                         mode="markers+text",
                         name="Early Access Eps",
                         text=ea_markers['ea_count'],
                         textposition="top center")

In [395]:
def count_cumulation(df, col_name, new_col_name):
    count = 0
    col_list = []
    
    for _, row in df.iterrows():
        count+=row[col_name]
        col_list.append(count)
    df[new_col_name] = col_list
    return df

purs = count_cumulation(purchases, 'total_revenue', 'sum_revenue')
purs = count_cumulation(purchases, 'early_access_revenue', 'sum_ea_revenue')
purs = count_cumulation(purchases, 'ios_subs_revenue', 'sum_ios_subs_revenue')
purs = count_cumulation(purchases, 'android_subs_revenue', 'sum_android_subs_revenue')
purs.head()

,purchaseDate,paid_users,all_purs,ios_subs,ios_rebills,android_subs,android_rebills,early_accesses,total_revenue,ios_subs_revenue,android_subs_revenue,early_access_revenue,sum_revenue,sum_ea_revenue,sum_ios_subs_revenue,sum_android_subs_revenue
0,2019-12-05,1,1,1,2,0,0,0,340.016,340.016,0,0,340.016,0,340.016,0
1,2019-12-06,7,7,7,12,0,0,0,2040.096,2040.096,0,0,2380.112,0,2380.112,0
2,2019-12-07,4,4,3,1,1,4,0,358.976,170.008,188.968,0,2739.088,0,2550.120,188.968
3,2019-12-08,4,4,3,0,1,2,0,188.968,0.000,188.968,0,2928.056,0,2550.120,377.936
4,2019-12-09,3,3,2,0,1,5,0,188.968,0.000,188.968,0,3117.024,0,2550.120,566.904


In [401]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

# fig.add_trace(go.Scatter(x=purs['purchaseDate'],
#                          y=purs['sum_revenue'],
#                          name='Revenue'),
#               secondary_y=True,)

fig.add_trace(go.Scatter(x=purs['purchaseDate'],
                         y=purs['sum_ea_revenue'],
                         name='EA Revenue',
                         stackgroup='one'),
              secondary_y=True,)

fig.add_trace(go.Scatter(x=purs['purchaseDate'],
                         y=purs['sum_ios_subs_revenue'],
                         name='iOS Subs Revenue',
                        stackgroup='one'),
              secondary_y=True,)

fig.add_trace(go.Scatter(x=purs['purchaseDate'],
                         y=purs['sum_android_subs_revenue'],
                         name='Android Subs Revenue',
                        stackgroup='one'),
              secondary_y=True,)

fig.add_trace(go.Scatter(x=merged_data['date'], 
                         y=merged_data['users_stop_reading'],
                         name="Users Stopped Reading"), secondary_y=False,)

fig.add_trace(published_eps_trace(merged_data['users_stop_reading'].min()), secondary_y=False,)
fig.add_trace(ea_markers_trace(merged_data['users_stop_reading'].max()), secondary_y=False,)

fig.show()

In [332]:
fig = px.line(merged_data, x="date", y="users_read", color="fin_eps", title="Users Stopped at Episode")
fig.add_trace(published_eps_trace(merged_data['users_read'].min()))
fig.add_trace(ea_markers_trace(merged_data['users_read'].max()))
fig.show()

In [382]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=merged_data["fin_eps"], 
    y=merged_data["users_read"],
    histfunc="sum",
    histnorm='percent'
))

fig.update_layout(
    title_text='Stopped At Episode', # title of plot
    xaxis_title_text='Episode', # xaxis label
    yaxis_title_text='Users, %', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    ),
    uniformtext_minsize=8, uniformtext_mode='hide'
)

fig.show()